In [1]:
# clear any existing session
from tensorflow.keras import backend as K
K.clear_session()

In [2]:
# imports
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from wandb.keras import WandbCallback
from utils import data_utils_v2
import tensorflow as tf
import numpy as np
import wandb
import time

In [3]:
# fix random seed for better reproducibility
tf.random.set_seed(666)

In [4]:
# Enable XLA
tf.config.optimizer.set_jit(True)

# Enable AMP
tf.keras.mixed_precision.experimental.set_policy('mixed_float16')

A brief introduction to XLA is available [here](https://docs.google.com/presentation/d/1F7hBey7m7bKSmLB4-Ipe9KvZl--TkaJGi69wRzzpAGM/edit#slide=id.p1). It helps to fuse certain operations (like addition, division, sqrt) used in a deep learning model thereby speeding up computation. 

In [5]:
# initialize wandb
wandb.init("ml-bootcamp")

W&B Run: https://app.wandb.ai/sayakpaul/ML-Bootcamp-Launchpad/runs/dao8p761

In [6]:
# don't change this
CLASSES = [b'daisy', b'dandelion', b'roses', b'sunflowers', b'tulips']

In [7]:
# define the constants
BATCH_SIZE = 80
EPOCHS = 20

In [8]:
# let's load up the tfrecord filenames
tfr_pattern_train = "train_tfr/*.tfrec"
train_filenames = tf.io.gfile.glob(tfr_pattern_train)
tfr_pattern_test = "test_tfr/*.tfrec"
test_filenames = tf.io.gfile.glob(tfr_pattern_test)

In [9]:
# create the train and test dataset
training_dataset, steps_per_epoch = data_utils_v2.batch_dataset(train_filenames, BATCH_SIZE, True)
validation_dataset, validation_steps = data_utils_v2.batch_dataset(test_filenames, BATCH_SIZE, False)

Let's create a utility function which would return us an adjusted ResNet50 model. 

In [10]:
def create_model(img_size=(224,224), num_class=5, train_base=True):
    input_layer = Input(shape=(img_size[0],img_size[1],3), dtype=tf.float16)
    base = VGG16(input_tensor=input_layer,
                    include_top=False,
                    weights="imagenet")
    base.trainable = train_base
    x = base.output
    x = GlobalAveragePooling2D()(x)
    
    preds = Dense(num_class, activation="softmax", dtype=tf.float32)(x)
    return Model(inputs=input_layer, outputs=preds)

In [11]:
# instantiate the model, supply the loss scaled optimizer,
# and compile it
model = create_model()
opt = Adam(learning_rate=1e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [12]:
# train the model
start = time.time()
model.fit_generator(training_dataset, 
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_dataset,
    validation_steps=validation_steps,
    epochs=EPOCHS,
    callbacks=[WandbCallback(data_type="image", labels=CLASSES)])
wandb.log({"training_time": time.time() - start})

Epoch 1/20
39/39 [==============================] - 49s 1s/step - loss: 1.1994 - accuracy: 0.5179 - val_loss: 0.5957 - val_accuracy: 0.7667
Epoch 2/20
39/39 [==============================] - 39s 997ms/step - loss: 0.6266 - accuracy: 0.7654 - val_loss: 0.3774 - val_accuracy: 0.8583
Epoch 3/20
39/39 [==============================] - 38s 966ms/step - loss: 0.4032 - accuracy: 0.8599 - val_loss: 0.2260 - val_accuracy: 0.9229
Epoch 4/20
39/39 [==============================] - 37s 957ms/step - loss: 0.3346 - accuracy: 0.8859 - val_loss: 0.3904 - val_accuracy: 0.8708
Epoch 5/20
39/39 [==============================] - 37s 959ms/step - loss: 0.3104 - accuracy: 0.8785 - val_loss: 0.2153 - val_accuracy: 0.9250
Epoch 6/20
39/39 [==============================] - 37s 960ms/step - loss: 0.2875 - accuracy: 0.8958 - val_loss: 0.1343 - val_accuracy: 0.9583
Epoch 7/20
39/39 [==============================] - 37s 952ms/step - loss: 0.2511 - accuracy: 0.9109 - val_loss: 0.1394 - val_accuracy: 0.9542
Ep

Model trains much faster and still it is performing :)

A comparative study on mixed precision training is available [here](https://github.com/sayakpaul/Mixed-Precision-Training-in-tf.keras-2.0).

We are going to export our Keras model to `SavedModel` format. But why this format?

> SavedModel is a standalone serialization format for TensorFlow objects, supported by TensorFlow serving as well as TensorFlow implementations other than Python. - [Save and serialize models with Keras](https://www.tensorflow.org/guide/keras/save_and_serialize)

`tf.keras` offers a lot of options to serialize models and this a great read in that line: https://www.tensorflow.org/guide/keras/save_and_serialize. 

First, let's convert the Keras model to TensorFlow estimator and then convert it to SavedModel format. `tf.keras` allows for direct conversion to `SavedModel` using `model.save(dir_name, save_format="tf")` but the input shapes of our model is not supported for that direct conversion. So, we need the extra step. 

*The warnings can be ignored*. We will now write a serving preprocessing function which will be appended to our model's graph helping us to serve predictions efficiently. 

In [23]:
# get the name of the first layer of the model
model_input_name = model.input_names[0]
model_input_name

'input_1'

In [24]:
# function courtesy: http://bit.ly/2YxFbzN
def serving_input_receiver_fn():
    # initialize a placeholder to recieve the image
    input_ph = tf.compat.v1.placeholder(tf.string, shape=[None], name='image_binary')
    # map the image with decode_image to cast it to uint8 dtype
    images = tf.map_fn(tf.image.decode_image, input_ph, dtype=tf.uint8)
    # cast the pixels to float32 and scale the values, and cast back 
    # to float16
    images = tf.cast(images, tf.float32) / 255.
    images = tf.cast(images, tf.float16)
    
    # set the dimensions (None because we don't know how many images would come)
    images.set_shape([None, 224, 224, 3])

    # 1st: what will be going into the model
    # 2nd: what comes to the model initially
    return tf.estimator.export.ServingInputReceiver({model_input_name: images}, {'bytes': input_ph})

In [26]:
# convert TF estimator to SavedModel
export_path = estimator_model.export_saved_model('saved_model', serving_input_receiver_fn=serving_input_receiver_fn)
export_path

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from estimator_model/keras/keras_model.ckpt
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_model/temp-b'1575801062'/saved_model.pb


b'saved_model/1575801062'